# Bob has a problem
## Exceptionally DO NOT RUN this one!
## DO NOT delete output!!
#### 06.9 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-16

* How to get rid of an annoying ASA

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig

import algosdk.error
import json
import base64
import pandas as pd

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


## Bob has a problem
* We cannot access the holdings of Bob in our nice function
* Because Bob holds zero units of an ASA that has been deleted (the TEMP coin)
* We want to find the culprit and get rid of it

In [7]:
# This produces the problem
Bob_holding=asset_holdings_df(algod_client, Bob['public'])

AlgodHTTPError: asset does not exist

#### Step 1: Identify the problematic ASA
* It must be an ASA with zero holdings
* And possibly missing information

In [ ]:
algod_client.account_info(Bob['public'])['assets']

#### Step 2: Get rid of the problematic ASA
* Send an `AssetTransferTxn` with the `close_assets_to` argument

In [16]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(0)
asset_index = 70637093


txn = AssetTransferTxn(sender=Bob['public'], 
                       sp=sp, 
                       receiver=Bob['public'], 
                       amt=amt,
                       index=asset_index, 
                       close_assets_to=Bob['public']
                      )

# Step 2+3+4: sign and send and wait ...
stxn = txn.sign(Bob['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19813781.
Waiting for round 19813781 to finish.
Waiting for round 19813782 to finish.
Transaction TXS74NE3OUBF5KXNKRFHAEDNR7QJPOEPIUB4Z2WZNZFB5IBWSJNA confirmed in round 19813783.
